In [2]:
import os

In [4]:
os.getcwd()
os.chdir("../")
os.getcwd()

In [107]:
%load_ext autoreload
%autoreload 2
from run_training_pipeline import (
download_data,
prepare_data,
instantiate_dataset,
instantiate_model,
instantiate_loss,
instantiate_trainer,
download_prepare_test
)

In [17]:
from train_pipeline_utils.download_data import load_donnees_test
from classes.data.satellite_image import SatelliteImage

In [110]:
list_data_dir

In [112]:
length_delete = 230
dir_path = "../data/PLEIADES/2022/MARTINIQUE/"

files = os.listdir(dir_path)[:length_delete]

for file in files:
    file_path = os.path.join(dir_path, file)
    if os.path.isfile(file_path):
        os.remove(file_path)


In [113]:
with open("../config.yml") as f:
    config = yaml.load(f, Loader=SafeLoader)

list_data_dir = download_data(config)
list_output_dir = prepare_data(config, list_data_dir)

download_prepare_test(config)
model = instantiate_model(config)

train_dl, valid_dl, test_dl = instantiate_dataloader(
    config, list_output_dir
)

